# Create LULC

In [317]:
%matplotlib inline
%load_ext chime

import os
from glob import glob
import pickle
import chime

import rasterio as rio
import gdal
import numpy as np
import pandas as pd
import geopandas as gpd
from skimage.morphology import binary_closing, binary_opening, square, remove_small_holes, disk, square
from Functions import readResampledWindow, getSentinelBandFile

from skimage.filters.rank import modal, mean_bilateral
from datetime import datetime, timedelta, time
from time import sleep
import nbimporter
#from KeyFunctions import *
import chime
from utils import *
from skimage.morphology import binary_closing, binary_opening, square, remove_small_holes, disk, square, remove_small_objects

import richdem as rd

import lightgbm as lgb

import psutil


def throttleProcessors(workDayStart, workDayEnd):
    startPause = time(*(map(int, workDayStart.split(':'))))
    endPause = time(*(map(int, workDayEnd.split(':'))))
    nowTime = datetime.today().time()
    if nowTime < endPause or nowTime > startPause:
        print(f"Current time is {nowTime}. Setting processor/thread use to 4")
        return 4
    else:
        return 8

def wait_start(workDayStart="8:00", workDayEnd="19:00", force=False):
    startPause = time(*(map(int, workDayStart.split(':'))))
    endPause = time(*(map(int, workDayEnd.split(':'))))
    nowTime = datetime.today().time()
    if nowTime < endPause and nowTime > startPause and datetime.today().isoweekday() <= 5 and not force:
        waitTimeSeconds = datetime.combine(datetime.today(), endPause) - datetime.combine(datetime.today(), nowTime)
    else:
        waitTimeSeconds = False
    return waitTimeSeconds


def createClassifiedRaster(classification_model, ortho_file, classifiedFiles_loc, suffix=None, binaryClass=None, overwrite=False):
    start = datetime.now()
    if suffix == None:
        suffix = datetime.now().strftime("%Y%m%d")
    daynum = 1
    output_image = os.path.join(classifiedFiles_loc, os.path.basename(ortho_file))#_{day}-{daynum}.tif"))
    if binaryClass:
        #print(f"Setting binary output ({output_image})")
        #output_image = output_image.replace(".tif", f"_{binaryClass}BinaryLGBNewishGBLM.tif")
        output_image = output_image.replace(".tif", f"_{binaryClass}BinaryOptunaTunerGBLM_{suffix}.tif")
    else:
        output_image = output_image.replace(".tif", f"_MultiClassOptunaTunerGBLM_{suffix}_v2.tif")
        
    if os.path.exists(output_image) and not overwrite:
        print(f"File exists ({output_image})")
        return output_image
    try:
        with rio.open(ortho_file) as src:
            kwargs = src.profile
            data = src.read()
            descs = list(src.descriptions)
    except:
        print(f"ERROR: Unable to open {os.path.basename(ortho_file)}. Skipping")
        return None
        
    if binaryClass:
        feature_names = classification_model.feature_name()
        dtype = rio.float32
    else:
        feature_names = classification_model.feature_name_
        dtype = rio.uint8
    
    bands = {desc:data[ib] for ib, desc in enumerate(descs)}
    
    #This is a training dataset not created with the others, but may be in the classifier. Create and add
    if "RGBNmean" in feature_names:
        #print("Adding RGBNmean")
        rgbnMean = np.nanmean(data[:4], axis=0).astype(data.dtype)
        bands["RGBNmean"] = rgbnMean
        
    if "RED_LHE" in feature_names or "GREEN_LHE" in feature_names or "BLUE_LHE" in feature_names or "NIR_LHE" in feature_names:
        path = os.path.basename(ortho_file).split("_")[0]
        row = os.path.basename(ortho_file).split("_")[1]
        print(f"PATH: {path}, ROW: {row}")
        localHisto_file = findFile(path, row, histoEqual_loc)
        with rio.open(localHisto_file) as src:
            for i, desc in enumerate(src.descriptions):
                bands[desc] = src.read(i+1)
                
    if "REDnessNorm" in feature_names or "GREENnessNorm" in feature_names or "BLUEnessNorm" in feature_names or "NIRnessNorm" in feature_names:
        bands["REDnessNorm"] = (bands["RED"] - np.nanmean(np.stack([bands["GREEN"], bands["BLUE"], bands["NIR"]]),axis=0)) / (bands["RED"] + np.nanmean(np.stack([bands["GREEN"], bands["BLUE"], bands["NIR"]]), axis=0))
        bands["GREENnessNorm"] = (bands["GREEN"] - np.nanmean(np.stack([bands["RED"], bands["BLUE"], bands["NIR"]]), axis=0)) / (bands["GREEN"] + np.nanmean(np.stack([bands["RED"], bands["BLUE"], bands["NIR"]]), axis=0))
        bands["BLUEnessNorm"] = (bands["BLUE"] - np.nanmean(np.stack([bands["GREEN"], bands["RED"], bands["NIR"]]), axis=0)) / (bands["BLUE"] + np.nanmean(np.stack([bands["GREEN"], bands["RED"], bands["NIR"]]), axis=0))
        bands["NIRnessNorm"] = (bands["NIR"] - np.nanmean(np.stack([bands["GREEN"], bands["BLUE"], bands["RED"]]), axis=0)) / (bands["NIR"] + np.nanmean(np.stack([bands["GREEN"], bands["BLUE"], bands["RED"]]), axis=0))
        
    
    for name in feature_names:
        if "Sentinel2_" in name:
            sentBand = name.split("_")[-1]
            sentFile = getSentinelBandFile(sentBand, "../Sentinel2Data/")
            bands[name] = readResampledWindow(sentFile, ortho_file, returnData=True)

    output = createClassificationFromModel(classification_model, bands, feature_names)

    #output = np.rint(output)
    # create our final mask
    #mask = (~m.mask[:,0]).reshape(*img_swp.shape[:-1])

    kwargs.update(dtype=dtype, count=1)
    with rio.open(output_image, 'w', **kwargs) as dst: 
        # write to the final file
        #dst.write(output.astype(rio.uint8), 1)
        dst.write(output.astype(dtype), 1)
        #dst.write_mask(mask)
        if not binaryClass:
            colors = {
                1: (12,42,235, 255),
                2: (41, 210, 219,255),
                3: (255, 214, 117, 255),
                4: (171, 224, 85, 255),
                5: (12, 100, 1, 255),
                6: (0, 192, 32, 255),
                7: (62, 62, 62, 255),
                8: (160, 160, 160, 255),
                9: (160, 37, 6, 255)
                }
            dst.write_colormap(1, colors)

    #print("WROTE IT")
    """if binaryClass:
        cleaned_loc = os.path.abspath(r"..\EPCExtent_30cm\Orthos_Segmentedv3_Classifiedv3_cleaned")
        cleanedOutput_loc = os.path.join(cleaned_loc, os.path.basename(ortho_file).replace(".tif","_cleaned.tif"))
        cleanedOutput = cleanIt(output, inData=bands, inClass=binaryClass, inprofile=kwargs)
        kwargs.update(nodata=0, dtype=rio.float32)
        with rio.open(cleanedOutput_loc, 'w', **kwargs) as dst:
            #dst.write(cleanedOutput.astype(rio.uint8), 1)
            dst.write(cleanedOutput.astype(rio.float32), 1)
    """

    print(f"Classified to {os.path.abspath(output_image)}. \nClassification took {datetime.now()-start}")

    return output_image
#except:
    print(f"Failed for {output_image}")


def createBinaryClassifiedRaster(models, ortho_file, classifiedFiles_loc, roundConfidence=False, cleanPixels=False, overwrite=False):
    memoryUsage = psutil.virtual_memory().percent
    waitStart = wait_start()
    if memoryUsage>50 and waitStart:
        print(f"Current time is {datetime.now().time()} on day # {datetime.today().weekday()} and memory usage at {memoryUsage}%. Waiting {round(waitStart.seconds/60/60,1)} hours")
        sleep(waitStart.seconds)
        print(f"Current time is {datetime.now().time()}. Resuming...")
        
    start = datetime.now()
    day = datetime.now().strftime("%Y%m%d")
    daynum = 1
    output_file = os.path.basename(ortho_file).replace(".tif", f"_BinaryOptunaTunerGBLM.tif")
    output_path = os.path.join(classifiedFiles_loc, output_file)#_{day}-{daynum}.tif"))
    cleaned_loc = os.path.abspath(r"..\EPCExtent_30cm\Orthos_Segmentedv3_Classifiedv3_cleaned")
    cleanedOutput_path = os.path.join(cleaned_loc, output_file.replace(".tif","_cleaned.tif"))
    
    if os.path.exists(output_path) and not overwrite:
        #print(f"File exists ({output_image})")
        return output_path
    
    try:
        with rio.open(ortho_file) as src:
            kwargs = src.profile
            kwargs.update(
                dtype= rio.uint8,
                count= 1,
            )
            data = src.read()
            descs = list(src.descriptions)
    except:
        print(f"ERROR: Unable to open {os.path.basename(ortho_file)}. Skipping")
        return None
    
    try:
        all_feature_names = set( fn for model_path in models.values() for fn in pickle.load(open(model_path, 'rb')).feature_name_ )
        #feature_names = classification_model.feature_name_
    except:
        all_feature_names = set( fn for model_path in models.values() for fn in pickle.load(open(model_path, 'rb')).feature_name() )
        #feature_names = classification_model.feature_name()
    

    bands = {desc:data[ib] for ib, desc in enumerate(descs)}
    
    #This is a training dataset not created with the others, but may be in the classifier. Create and add
    if "RGBNmean" in all_feature_names:
        #print("Adding RGBNmean")
        rgbnMean = np.nanmean(data[:4], axis=0).astype(data.dtype)
        bands["RGBNmean"] = rgbnMean
        
    if "RED_LHE" in all_feature_names:
        path = os.path.basename(ortho_file).split("_")[0]
        row = os.path.basename(ortho_file).split("_")[1]
        print(f"PATH: {path}, ROW: {row}")
        localHisto_file = findFile(path, row, histoEqual_loc)
        with rio.open(localHisto_file) as src:
            for i, desc in enumerate(src.descriptions):
                bands[desc] = src.read(i+1)
    
    for name in all_feature_names:
        if "Sentinel2_" in name:
            sentBand = name.split("_")[-1]
            sentFile = getSentinelBandFile(sentBand, "../Sentinel2Data/")
            bands[name] = readResampledWindow(sentFile, ortho_file, returnData=True)
    try:
        if roundConfidence:
            dtype = np.uint8
        else:
            dtype = np.float16
        kwargs.update(
            dtype = dtype,
            count = 9,
            nodata = 0
        )
        with rio.open(output_path, 'w', **kwargs) as dst: 
            for i, class_name in enumerate(models):
                model = pickle.load(open(models[class_name], 'rb'))
                confidenceClassification = createClassificationFromModel(model, all_bands=bands).astype(np.uint8)
                # Model provides confidence values from 0 to 1, round to binary if specifid
                if roundConfidence:
                    binaryClassification = np.rint(binaryClassification)
                # write to the final file
                dst.write(binaryClassification.astype(dtype), i+1)
                dst.set_band_description(i+1, class_name)

                if cleanPixels and roundConfidence:
                    cleanedOutput = cleanIt(binaryClassification==1, inData=bands, inClass=class_name)
                    with rio.open(cleanedOutput_path, "w", **kwargs) as cdst:
                        cdst.write(cleanedOutput.astype(dtype), i+1)
                        cdst.set_band_description(i+1, class_name)


        print(f"Classified to {os.path.abspath(output_path)}. \nClassification took {datetime.now()-start}")
    except:
        print(f"Failed on {output_path}. Returning None")
        try:
            os.remove(output_path)
        except:
            print(f"Unable to remove {output_path}")
            
        output_path = None
    
    return output_path


def createClassificationFromModel(model, all_bands, model_features):
        
    features = {fn: all_bands[fn] for fn in model_features}# if fn in descs}
    featureArrays = list(features.values())
    
    trainingFeatures = np.stack(featureArrays)

    #print("Input Data shape", data.shape)

    # read the image into the proper format, adding indices if necessary
    img_swp = np.moveaxis(trainingFeatures, 0, -1)
    img_flat = img_swp.reshape(-1, img_swp.shape[-1])

    #flatten bands along axis
    img_flat = img_swp.reshape(-1, img_swp.shape[-1])

    # remove no data values, store the indices for later use
    # a later cell makes the assumption that all bands have identical no-data value arrangements
    m = np.ma.masked_invalid(img_flat)
    to_predict = img_flat[~m.mask].reshape(-1, img_flat.shape[-1])
    #print("TO PREDICT SHAPE", to_predict.shape)

    # predict
    #print("Beginning prediction...", datetime.now())
    img_preds = model.predict(to_predict)

    #img_preds = [np.argmax(x) for x in img_preds]

    # add the prediction back to the valid pixels (using only the first band of the mask to decide on validity)
    # resize to the original image dimensions
    output = np.zeros(img_flat.shape[0])
    output[~m.mask[:,0]] = img_preds#.flatten()
    output = output.reshape(*img_swp.shape[:-1])

    return output


def cleanupStructures(array, value):
    structure = array == value
    array[structure] = 0
    for i in range(3):
        structure = binary_opening(structure, square(3))
        structure = binary_closing(structure, square(5))
    
    array[structure] = value
    array = fillHolesInClass(array, value, 90)
    array = np.where(structure==1, value, array)
    
    #for i in range(5):
    #    print(f"{i} iteration"
    #non_structure_mode = modal(array, square(21), mask=array==9)
    #array = np.where(array==255, non_structure_mode, array)

    return array

    
def cleanIt(inarray, inData, inClass, inprofile):
    if inClass=="Pool":
        outarray = cleanupPool(inarray, inData)
    elif inClass=="Asphault":
        outarray = cleanupAsphault(inarray, inprofile)
    elif inClass=="DenseVeg":
        outarray = cleanupDenseVeg(inarray, inData)
    elif inClass=="SparseVeg":
        outarray = cleanupSparseVeg(inarray, inData)
    elif inClass=="PondLake":
        outarray = cleanupPondLake(inarray, inData)
    else:
        outarray = inarray
        
    return outarray


def classifyTiles(df, model, forceStart=False, overwrite=False, binaryClass=None):
    print(f"\n\nStarting processing for {len(df)} tiles\n")
    classifiedFiles = []
    for i, r in df.iterrows():
        waitTime = wait_start(force=forceStart)
        if waitTime:
            buildVRT(classifiedFiles_loc, "EPC_30cmOrthoSegmented_Classified.vrt")
            pause_until = datetime.now() + waitTime
            print(f"waiting {timedelta(seconds=waitTime.seconds)} until {pause_until.strftime('%H:%m')}")
            sleep(waitTime.seconds)
            
        print(f"Starting {r.path}_{r.row} @ {datetime.now()}")
        #try:
        result = createClassifiedRaster(classification_model=model,
                                            ortho_file=r.OrthoFile,
                                            classifiedFiles_loc=classifiedFiles_loc,
                                            binaryClass=binaryClass,
                                            overwrite=overwrite)
        classifiedFiles.append(result)
        #except:
        #    print(f"FAILED FOR {r.OrthoFile}")

    return classifiedFiles


def fillHolesInClass(array, class_num, size_max):
    class_bool = array == class_num
    array[class_bool] = 255
    filled = remove_small_holes(class_bool, area_threshold=size_max, connectivity=1)
    array[filled] = class_num
    return array


def smoothValue(array, value, selem=square(5)):
    onlys = array == value
    for i in range(2):
        onlys = binary_closing(onlys, selem)
    array[onlys] = value
    #array[~asphault] = 255
    return array


def cleanupAsphault(a, inprofile):
    # 1. Drop where any slope percentage is less than 10 degrees
    slope = getSlopeArray(inprofile)
    slope_thres = np.where(slope<10, 0, 1)
    slope_thres = remove_small_objects(slope_thres, min_size=1000, connectivity=1)
    a[slope_thres==1] = 0
    
    # 2. Burn in mix-used paths
    
    # 3. Remove groups of asphault pixels smaller than 1000
    a = remove_small_objects(a==1, min_size=1000, connectivity=1)
    
    # 4. Fill in holes
    a = fillHolesInClass(a, 9, 90)
    
    # 5. Open and Dilate and to smooth edges
    for i in range(5):
        a = binary_opening(a, square(3))
        a = binary_closing(a, square(5))
        
    return a 

def cleanupImpervious(a, indata):
    a[indata["MSAVI"] > 30000] = 0
    
    return a

def cleanupPool(a, indata):
    a = remove_small_objects(a==1, min_size=50, connectivity=1)
    a = remove_small_holes(a==1, area_threshold=5, connectivity=1)
    for i in range(2):
        a = binary_opening(a, disk(2))
    
    a[indata["HAG"] > 8] = 0
    a[indata["NIR"] > 10000] = 0
    a[indata["BLUEness"] < 35000] = 0
    
    return a


def cleanupPondLake(a, indata):
    # drop ponds/lakes smaller than x?
    #a = removeClassSmaller(a, 1, 10000) 
    ndvi_lim = indata["NDVI"]<12000
    ndpi_lim = indata["NDPI"]>26000
    blueness_lim = indata["BLUEness"]<=40000
    a[ndvi_lim & ndpi_lim & blueness_lim] = 1
    a[a==1 & (~ndvi_lim | ~ndpi_lim | ~blueness_lim)] = 0
    a = remove_small_objects(a==1, min_size=5000, connectivity=1)
    # smooth ponds
    a = smoothValue(a, 1, selem=disk(5))

    return a

def cleanupIrrigated(a, value, indata=None):
     # set very green veg with high index and low brightness to irrigated
    a = remove_small_holes(a==1, area_threshold=5, connectivity=1)
    a = smoothValue(a, value, selem=disk(5))
    #a[(indata["MSAVI"]>=29556)
    #     & (indata["GREENness"]>=34307)
    #     & (indata["RGBNmean"]<=19273) ] = 1
    
    return a
        
def cleanupDenseVeg(a, value):
    a = smoothValue(a, value, selem=disk(2)) # smooth dense veg
    return a


def cleanupSparseVeg(a, value):
    a = smoothValue(a, value, selem=square(5)) # smooth sparse veg
    return a


def getSlopeArray(tprofile):
    with rio.open(r"D:/EPC_DEM_2015.vrt") as src:
        intrans = tprofile["transform"]
        inheight = tprofile["height"]
        inwidth = tprofile["width"]
        inres = intrans.a
        win = from_bounds(intrans.c, intrans.f-(inheight*inres), intrans.c+(inwidth*inres), intrans.f, src.transform).round_offsets().round_lengths()
        
        if win.col_off < src.width and win.row_off < src.height:
            dem = src.read(1, window=win, out_shape=(tprofile["height"], tprofile["width"]))
            slope = rd.TerrainAttribute(rd.rdarray(dem, no_data=src.nodata), attrib="slope_degrees")
            # if the read window includes an area with no-data, that means that it's outside of the region for the 2015 2ft DEM. Fill in with NED data
            if src.nodata in dem:
                slope10m = get10mNEDSlope(intrans,inheigh,inwidth)
                slope[dem==src.nodata] = slope10m
        else:
            # the entire window is outside of the extent of the 2ft DEM coverage; use NED
            slope = get10mNEDSlope(intrans,inheight,inwidth)

    return slope


def get10mNEDSlope(intrans, inheight, inwidth):
    inres = intrans.a
    with rio.open("../OtherData/10mDEMs/DEM10mNED_slope.tif") as src10m:
        win = from_bounds(intrans.c, intrans.f-(inheight*inres), intrans.c+(inwidth*inres), intrans.f, src10m.transform).round_offsets().round_lengths()
        slope = src10m.read(1, window=win, out_shape=(inheight, inwidth))
        
    return slope


def burnShadows(a, indata):
    # burn in shadows where low values, but not pool or pond
    rgbnMean = np.nanmean([indata["RED"], indata["GREEN"], indata["BLUE"], indata["NIR"]], axis=0)
    a[(rgbnMean<10282) & (a!=1) & (a!=2)] = 10
    
    return a


def getBandByDescription(file, bandDesc):
    if type(bandDesc) == list:
        getBands = bandDesc[:]
    elif type(bandDesc) == str:
        getBands = [bandDesc]
        
    bandDict = {}
    with rio.open(file) as src:
        for band in getBands:
            try:
                index = src.descriptions.index(band)
            except:
                raise ValueError(f"Unable to find band {band} in {file}")    
            # have to increment index since tiff indicies start at 1 and not 0
            bandDict[band] = src.read(index+1)

    if type(bandDesc) != list:
        return bandDict[bandDesc]
    
    return bandDict

The chime extension is already loaded. To reload it, use:
  %reload_ext chime


In [3]:
classifiedFiles_loc = "../EPCExtent_30cm/Orthos_Segmented_Classifiedv3"
binaryClassifiedFiles_loc = "../EPCExtent_30cm/Orthos_Segmented_Classifiedv3_Binary"
histoEqual_loc = os.path.abspath("../EPCExtent_30cm/Orthos_LocalHistogramEqualized")

#propsDir = r"../EPCExtent_30cm/Orthos_Segmentedv2_properties"
orthosDir = r"../EPCExtent_30cm/Orthos_Segmentedv3"
os.makedirs(classifiedFiles_loc, exist_ok=True)

tindex = gpd.read_file("../EPCExtent_30cm/Ortho_5kSubIndex.gpkg")
to_process = gpd.read_file("../temp/ToProcess.gpkg")

print(f"{len(tindex)} total tile indicies")
tindex["OrthoFile"] = tindex.apply(lambda r: findFile(path=r.path, row=r.row, directory=orthosDir), axis=1)
#tindex["PropsFile"] = tindex.apply(lambda r: findFile(path=r.path, row=r.row, files=propsDir), axis=1)

# ignore tiles which don't have input variables created
tindex = tindex[(~pd.isnull(tindex.OrthoFile))]
print(f"{len(tindex)} tile indicies with training data already built")


#prioritize central tucson and work out from there
tindex["centroid"] = tindex.geometry.centroid
central_tile = tindex[(tindex.path == "W1004789") & (tindex.row == "W449850")]
central_point = central_tile.centroid.values[0]
tindex["DistToCenter"] = tindex.centroid.apply(lambda c: int(c.distance(central_point)))
tindex.sort_values(by="DistToCenter", inplace=True)

classifiers = {mp.split("_")[0].split("Binary")[-1]:mp for mp in glob("./Models/*20211117.sav")}

tt = tindex[((tindex.path == "W989789") & (tindex.row == "W439850")) |
            ((tindex.path == "W1004789") & (tindex.row == "W449850")) |
            ((tindex.path == "W1004789") & (tindex.row == "W344850")) |
            ((tindex.path == "W979789") & (tindex.row == "W419850")) |
            ((tindex.path == "W1004789") & (tindex.row == "W444850")) |
            ((tindex.path == "W919789") & (tindex.row == "W404850")) 
           ]

C:\Users\BenJames\anaconda3\envs\geospatial\lib\site-packages\geopandas\geodataframe.py:294: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for f in features_lst:


4426 total tile indicies
1783 tile indicies with training data already built


## Binary Group Parallelized

In [4]:
classifiers["Asphault"] = r"M:\PAG2019\notebooks\Models\lightGBMBinaryAsphault_LGBMTuner_20211115.sav"
classifiers["Pool"] = r"M:\PAG2019\notebooks\Models\lightGBMBinaryPool_LGBMTuner_20211115.sav"

In [70]:
%%time
asphaultFiles = Parallel(n_jobs=2)(delayed(createClassifiedRaster)(pickle.load(open(classifiers["Asphault"], "rb")),
                                                                   ortho_file= row.OrthoFile,
                                                                   classifiedFiles_loc= binaryClassifiedFiles_loc,
                                                                   binaryClass= "Asphault",
                                                                   suffix = "20211115",
                                                                   overwrite= False) for i, row in tindex.iterrows())

Wall time: 17.9 s


In [71]:
%%time
poolFiles = Parallel(n_jobs=2)(delayed(createClassifiedRaster)(pickle.load(open(classifiers["Pool"], "rb")),
                                                               ortho_file= row.OrthoFile,
                                                               classifiedFiles_loc= binaryClassifiedFiles_loc,
                                                               binaryClass= "Pool",
                                                               suffix = "20211115",
                                                               overwrite= False) for i, row in tindex.iterrows())

Wall time: 1min 3s


In [81]:
%%time
imperviousFiles = Parallel(n_jobs=2)(delayed(createClassifiedRaster)(pickle.load(open("./Models/lightGBMBinaryImpervious_OptunaTuner_20211115.sav", "rb")),
                                                                     ortho_file= row.OrthoFile,
                                                                     classifiedFiles_loc= binaryClassifiedFiles_loc,
                                                                     binaryClass= "Impervious",
                                                                     suffix= "20211115",
                                                                     overwrite= False) for i, row in tt.iterrows())
%chime
[os.path.abspath(f) for f in imperviousFiles]

Wall time: 6min 37s


['M:\\PAG2019\\EPCExtent_30cm\\Orthos_Segmented_Classifiedv3_Binary\\W1004789_W449850_TrainingStackV3_ImperviousBinaryOptunaTunerGBLM_20211130.tif',
 'M:\\PAG2019\\EPCExtent_30cm\\Orthos_Segmented_Classifiedv3_Binary\\W1004789_W444850_TrainingStackV3_ImperviousBinaryOptunaTunerGBLM_20211130.tif',
 'M:\\PAG2019\\EPCExtent_30cm\\Orthos_Segmented_Classifiedv3_Binary\\W989789_W439850_TrainingStackV3_ImperviousBinaryOptunaTunerGBLM_20211130.tif',
 'M:\\PAG2019\\EPCExtent_30cm\\Orthos_Segmented_Classifiedv3_Binary\\W979789_W419850_TrainingStackV3_ImperviousBinaryOptunaTunerGBLM_20211130.tif',
 'M:\\PAG2019\\EPCExtent_30cm\\Orthos_Segmented_Classifiedv3_Binary\\W919789_W404850_TrainingStackV3_ImperviousBinaryOptunaTunerGBLM_20211130.tif',
 'M:\\PAG2019\\EPCExtent_30cm\\Orthos_Segmented_Classifiedv3_Binary\\W1004789_W344850_TrainingStackV3_ImperviousBinaryOptunaTunerGBLM_20211130.tif']

In [86]:
imperviousFiles = glob("../EPCExtent_30cm/Orthos_Segmented_Classifiedv3_Binary/*_TrainingStackV3_ImperviousBinaryOptunaTunerGBLM_20211115.tif")
barrenFiles = glob("../EPCExtent_30cm/Orthos_Segmented_Classifiedv3_Binary/*_TrainingStackV3_BarrenBinaryOptunaTunerGBLM_20211115.tif")

(1783, 5)

In [179]:
barrenFiles = glob("../EPCExtent_30cm/Orthos_Segmented_Classifiedv3_Binary/*_TrainingStackV3_Barren*.tif")
len(barrenFiles)

23

In [182]:
%%time
barrenFiles = Parallel(n_jobs=1)(delayed(createClassifiedRaster)(pickle.load(open("Models\lightGBMBinaryBarren_LGBMTunerOptimum_20211130.sav", "rb")),
                                                                 ortho_file= row.OrthoFile,
                                                                 classifiedFiles_loc= binaryClassifiedFiles_loc,
                                                                 binaryClass= "Barren",
                                                                 suffix = "20211130",
                                                                 overwrite= False) for i, row in tindex.iterrows())
%chime
[os.path.abspath(f) for f in barrenFiles]

File exists (../EPCExtent_30cm/Orthos_Segmented_Classifiedv3_Binary\W1004789_W449850_TrainingStackV3_BarrenBinaryOptunaTunerGBLM_20211130.tif)
File exists (../EPCExtent_30cm/Orthos_Segmented_Classifiedv3_Binary\W1004789_W454850_TrainingStackV3_BarrenBinaryOptunaTunerGBLM_20211130.tif)
File exists (../EPCExtent_30cm/Orthos_Segmented_Classifiedv3_Binary\W999789_W449850_TrainingStackV3_BarrenBinaryOptunaTunerGBLM_20211130.tif)
File exists (../EPCExtent_30cm/Orthos_Segmented_Classifiedv3_Binary\W1004789_W444850_TrainingStackV3_BarrenBinaryOptunaTunerGBLM_20211130.tif)
File exists (../EPCExtent_30cm/Orthos_Segmented_Classifiedv3_Binary\W1009789_W449850_TrainingStackV3_BarrenBinaryOptunaTunerGBLM_20211130.tif)
File exists (../EPCExtent_30cm/Orthos_Segmented_Classifiedv3_Binary\W999789_W444850_TrainingStackV3_BarrenBinaryOptunaTunerGBLM_20211130.tif)
File exists (../EPCExtent_30cm/Orthos_Segmented_Classifiedv3_Binary\W999789_W454850_TrainingStackV3_BarrenBinaryOptunaTunerGBLM_20211130.tif)
Fi

KeyboardInterrupt: 

In [83]:
%%time
pondLakeFiles = Parallel(n_jobs=2)(delayed(createClassifiedRaster)(pickle.load(open('./Models\\lightGBMBinaryPondLake_LGBMTuner_20211118.sav', "rb")),
                                                                     ortho_file= row.OrthoFile,
                                                                     classifiedFiles_loc= binaryClassifiedFiles_loc,
                                                                     binaryClass= "PondLake",
                                                                     suffix = "20211115",
                                                                     overwrite= False) for i, row in tindex.iterrows())


#%chime
#[os.path.abspath(f) for f in pondLakeFiles]

Wall time: 1min 27s


In [307]:
%%time
cleanBinaryDir = r"M:\PAG2019\EPCExtent_30cm\Orthos_Segmented_Classifiedv3_Binary_cleaned"

def getCleanedBinary(file, orthoFile, outdir, lcClass, probLimit=0.01, overwrite=False, writeOut=False):
    ofile = os.path.join(outdir, os.path.basename(file).replace(".tif", "_clean.tif"))
    #if os.path.exists(ofile) and not overwrite and not returnData:
    #    return ofile
    if os.path.exists(ofile) and not overwrite:
        data_clean = rio.open(ofile).read(1)
    else:
        with rio.open(file) as src:
            data = src.read(1)
            kwargs = src.profile
        
        #orthoFile = glob(orthosDir + "/" + "_".join(os.path.basename(file).split("_")[:2]) + "*.tif")[0]

        data = np.where(data>probLimit, 1, 0)

        if lcClass == "Asphault":
            data_clean = cleanupAsphault(data, kwargs)
        elif lcClass == "Pool":
            print("Cleaning Pool")
            trainingData = getBandByDescription(orthoFile, ["HAG","NIR","BLUEness"])
            data_clean = cleanupPool(data, trainingData)
            print("Cleaned Pool")
        elif lcClass == "PondLake":
            trainingData = getBandByDescription(orthoFile, ["NDVI","NDPI","BLUEness"])
            data_clean = cleanupPondLake(data, trainingData)
        elif lcClass == "Irrigated":
            trainingData = getBandByDescription(orthoFile, ["MSAVI","GREENness","RGBNmean"])
            data_clean = cleanupIrrigated(data, trainingData)
        else:
            raise ValueError(f"Unknown class type {lcClass}")
    
        if writeOut:
            kwargs.update(dtype=np.uint8, nodata=0)
            with rio.open(ofile, "w", nbits=1, **kwargs) as dst:
                dst.write(data_clean.astype(np.uint8), 1)
            return ofile
    
    return data_clean.astype(np.uint8)

Wall time: 0 ns


In [183]:
barrenFiles = glob("../EPCExtent_30cm/Orthos_Segmented_Classifiedv3_Binary/*_TrainingStackV3_BarrenBinaryOptunaTunerGBLM_20211130.tif")

tindex["BarrenFile"] = tindex.apply(lambda r: [f for f in barrenFiles if f != None and r.path in os.path.basename(f) and r.row in os.path.basename(f)], axis=1)
tindex["BarrenFile"] = tindex["BarrenFile"].apply(lambda f: f[0] if len(f) == 1 else None)

In [87]:
binaryStackFiles = glob("../EPCExtent_30cm/Orthos_Segmented_Classifiedv3/*_TrainingStackV3_BinaryOptunaTunerGBLM.tif")
tindex["AsphaultFile"] = tindex.apply(lambda r: [f for f in asphaultFiles if f != None and r.path in os.path.basename(f) and r.row in os.path.basename(f)][0], axis=1)
tindex["PoolFile"] = tindex.apply(lambda r: [f for f in poolFiles if f != None and r.path in os.path.basename(f) and r.row in os.path.basename(f)][0], axis=1)
tindex["PondLakeFile"] = tindex.apply(lambda r: [f for f in pondLakeFiles if f != None and r.path in os.path.basename(f) and r.row in os.path.basename(f)][0], axis=1)
tindex["ImperviousFile"] = tindex.apply(lambda r: [f for f in imperviousFiles if f != None and r.path in os.path.basename(f) and r.row in os.path.basename(f)][0], axis=1)
tindex["BarrenFile"] = tindex.apply(lambda r: [f for f in barrenFiles if f != None and r.path in os.path.basename(f) and r.row in os.path.basename(f)][0], axis=1)
tindex["BinaryStackFile"] = tindex.apply(lambda r: [f for f in binaryStackFiles if f != None and r.path in os.path.basename(f) and r.row in os.path.basename(f)][0], axis=1)
%chime

In [121]:
tindex["AsphaultFile"] = tindex["AsphaultFile"].apply(lambda f: f[0])
tindex["PoolFile"] = tindex["PoolFile"].apply(lambda f: f[0])
tindex["PondLakeFile"] = tindex["PondLakeFile"].apply(lambda f: f[0])
tindex["ImperviousFile"] = tindex["ImperviousFile"].apply(lambda f: f[0])
tindex["BinaryStackFile"] = tindex["BinaryStackFile"].apply(lambda f: f[0])

In [323]:
%%time
values_2019_lookup = {'PondLake': 1, 'Pool': 2, 'Barren': 3, 'SparseVeg': 4, 'DenseVeg': 5,
                      'IrrigatedLand': 6, 'Asphault': 7, 'Impervious': 8, 'Structure': 9}
                      
def buildCleanLULC(r, outdir, overwrite=False):
        #try:
        ofile = f"{r.path}_{r.row}_TrainingStackV3_BinaryStack_cleaned.tif"
        output_lulc = os.path.join(outdir, ofile)
        if os.path.exists(output_lulc) and not overwrite:
            return output_lulc

        with rio.open(r.OrthoFile) as src:
            kwargs = src.profile

        asphault = getCleanedBinary(r.AsphaultFile, r.OrthoFile, cleanBinaryDir, "Asphault", writeOut=False)
        pool = getCleanedBinary(r.PoolFile, r.OrthoFile, cleanBinaryDir, "Pool", writeOut=False)
        pondLake = getCleanedBinary(r.PondLakeFile, r.OrthoFile, cleanBinaryDir, "PondLake", probLimit=0.5, writeOut=False)

        binaryFile = r.BinaryStackFile
        
        irrigatedLand = cleanupIrrigated(getBandByDescription(binaryFile, bandDesc="IrrigatedLand"), 1)
        structures = cleanupStructures(getBandByDescription(binaryFile, bandDesc="Structure"), 1)
        sparseVeg = cleanupSparseVeg(getBandByDescription(binaryFile, bandDesc="SparseVeg"), 1)
        denseVeg = cleanupDenseVeg(getBandByDescription(binaryFile, bandDesc="DenseVeg"), 1)

        lulc_array = np.zeros(asphault.shape, dtype=np.uint8)
        lulc_array[structures == 1] = values_2019_lookup["Structure"]
        lulc_array[pool == 1] = values_2019_lookup["Pool"]
        lulc_array[pondLake == 1] = values_2019_lookup["PondLake"]
        lulc_array[irrigatedLand == 1] = values_2019_lookup["IrrigatedLand"]
        lulc_array[sparseVeg == 1] = values_2019_lookup["SparseVeg"]
        lulc_array[asphault == 1] = values_2019_lookup["Asphault"]
        lulc_array[denseVeg == 1] = values_2019_lookup["DenseVeg"]

        with rio.open(r.ImperviousFile) as src:
            impervious = src.read(1)
            impervious = cleanupImpervious(impervious, getBandByDescription(r.OrthoFile, ["MSAVI"]))
        with rio.open(r.BarrenFile) as src:
            barren = src.read(1)

        # stack barren and impervious and pick class with best confidence (highest/argmax)
        barrenImp = np.stack([impervious, barren])
        bestIndex = np.argmax(barrenImp, axis=0)
        lulc_array[(lulc_array==0) & (bestIndex==0)] = values_2019_lookup["Impervious"]
        lulc_array[(lulc_array==0) & (bestIndex==1)] = values_2019_lookup["Barren"]

        # burn in shadows where low values, but not pool or pond
        #lulc_array = burnShadows(lulc_array, getBandByDescription(r.OrthoFile, ["RED", "GREEN", "BLUE", "NIR"]))
        
        kwargs.update(count=1, dtype=np.uint8, nodata=0)
        with rio.open(output_lulc, 'w', **kwargs) as dst: 
            # write to the final file
            dst.write(lulc_array, 1)
            colors = {
                1: (12,42,235, 255),
                2: (41, 210, 219,255),
                3: (255, 214, 117, 255),
                4: (171, 224, 85, 255),
                5: (12, 100, 1, 255),
                6: (0, 192, 32, 255),
                7: (62, 62, 62, 255),
                8: (160, 160, 160, 255),
                9: (160, 37, 6, 255),
                10: (0, 0, 0, 255)
                }
            dst.write_colormap(1, colors)
        return output_lulc
        #except:
        #    print(f"Failed for {r.OrthoFile}")

        
tt = tindex[
            ((tindex.path == "W1004789") & (tindex.row == "W444850"))         
            | ((tindex.path == "W989789") & (tindex.row == "W439850"))
            | ((tindex.path == "W1004789") & (tindex.row == "W449850")) 
            | ((tindex.path == "W1004789") & (tindex.row == "W344850"))
            | ((tindex.path == "W979789") & (tindex.row == "W419850")) 
            | ((tindex.path == "W919789") & (tindex.row == "W404850")) 
           ]
tt["BarrenFile"] = tt.apply(lambda r: [f for f in barrenFiles if f != None and r.path in os.path.basename(f) and r.row in os.path.basename(f)][0], axis=1)

tt = tindex.dropna()
cleanLULCFiles = Parallel(n_jobs=8)(delayed(buildCleanLULC)(row, cleanBinaryDir, overwrite=False) for i, row in tt.iterrows())
#for i, row in tt.iterrows():
#    lulcFile = buildCleanLULC(row, cleanBinaryDir, overwrite=True)
#    print(os.path.abspath(lulcFile))
for f in cleanLULCFiles:
    print(f)
%chime

<timed exec>:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


M:\PAG2019\EPCExtent_30cm\Orthos_Segmented_Classifiedv3_Binary_cleaned\W1004789_W449850_TrainingStackV3_BinaryStack_cleaned.tif
M:\PAG2019\EPCExtent_30cm\Orthos_Segmented_Classifiedv3_Binary_cleaned\W1004789_W454850_TrainingStackV3_BinaryStack_cleaned.tif
M:\PAG2019\EPCExtent_30cm\Orthos_Segmented_Classifiedv3_Binary_cleaned\W999789_W449850_TrainingStackV3_BinaryStack_cleaned.tif
M:\PAG2019\EPCExtent_30cm\Orthos_Segmented_Classifiedv3_Binary_cleaned\W1004789_W444850_TrainingStackV3_BinaryStack_cleaned.tif
M:\PAG2019\EPCExtent_30cm\Orthos_Segmented_Classifiedv3_Binary_cleaned\W1009789_W449850_TrainingStackV3_BinaryStack_cleaned.tif
M:\PAG2019\EPCExtent_30cm\Orthos_Segmented_Classifiedv3_Binary_cleaned\W999789_W444850_TrainingStackV3_BinaryStack_cleaned.tif
M:\PAG2019\EPCExtent_30cm\Orthos_Segmented_Classifiedv3_Binary_cleaned\W999789_W454850_TrainingStackV3_BinaryStack_cleaned.tif
M:\PAG2019\EPCExtent_30cm\Orthos_Segmented_Classifiedv3_Binary_cleaned\W1009789_W454850_TrainingStackV3_Bin

In [332]:
t = gdal.BuildVRT("C:/Users/BenJames/Downloads/Cleaned2019LULC.vrt", cleanLULCFiles)
del t

In [331]:
buildVRT?

Signature: buildVRT(directory, outfile)
Docstring: <no docstring>
File:      m:\pag2019\notebooks\utils.py
Type:      function


In [39]:
%%time
cleanAsphaultFiles = Parallel(n_jobs=8)(delayed(createCleanAsphault)(file, cleanBinaryDir, overwrite=False) for file in asphaultFiles)
cleanPoolFiles = Parallel(n_jobs=8)(delayed(createCleanPool)(file, cleanBinaryDir, overwrite=False) for file in poolFiles)

C:\Users\BenJames\anaconda3\envs\geospatial\lib\site-packages\joblib\externals\loky\process_executor.py:703: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Wall time: 1h 25min 3s


### Testing for binary on tt

In [3]:
%%time
files = Parallel(n_jobs=2)(delayed(createBinaryClassifiedRaster)(models=classifiers,
                                                                 ortho_file=row.OrthoFile,
                                                                 classifiedFiles_loc=classifiedFiles_loc,
                                                                 overwrite=False) for i, row in tindex.iterrows())


Wall time: 2.27 s


In [74]:
count = 0
cutoff = datetime.strptime("2021-11-05 15:00", "%Y-%m-%d %H:%M")
files = glob("../EPCExtent_30cm//Orthos_Segmented_Classifiedv3/*.tif")
for file in files:
    filedate = datetime.fromtimestamp(os.path.getctime(file))
    delta = filedate-cutoff
    if delta.days <= -1:
        #os.remove(file)
        count +=1
        #print(file, delta)

print(count)

0


In [63]:
lbounds, kwargs["transform"].c, kwargs["transform"].f

(BoundingBox(left=984839.591874, bottom=449801.1852379983, right=989838.6077820001, top=454800.2011459983),
 984839.591874,
 454800.2011459983)

In [89]:
with rio.open("../EPCExtent_30cm/Orthos_Segmented_Classifiedv3_Binary/W984789_W454850_TrainingStackV3_AsphaultBinaryOptunaTunerGBLM_20211115.tif") as src:
    t = src.read(1)
    kwargs = src.profile
kwargs.update(dtype=np.uint8, nodata=0)
t = np.where(t>0.01, 1, 0)
tt = cleanupAsphault(t, kwargs)

with rio.open("C:/Users/BenJames/Downloads/W984789_W454850_TrainingStackV3_AsphaultBinaryOptunaTunerGBLM_20211115_clean2.tif", "w", **kwargs) as dst:
    dst.write(tt.astype(np.uint8), 1)

Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)


<ipython-input-88-bc4929200da8>:391: UserWarning: Only one label was provided to `remove_small_objects`. Did you mean to use a boolean array?
  slope_thres = remove_small_objects(slope_thres, min_size=1000, connectivity=1)


## Multilabel Group Parallized

### Testing for multilabel on tt

In [29]:
multiClassModel = pickle.load(open(r"M:\PAG2019\notebooks\Models\lightGBM_20211129_optimum.sav", "rb"))

In [69]:
%%time
files = Parallel(n_jobs=2)(delayed(createClassifiedRaster)(multiClassModel,
                                                           ortho_file=row.OrthoFile,
                                                           classifiedFiles_loc=classifiedFiles_loc,
                                                           binaryClass= None,
                                                           suffix = "20211129",
                                                           overwrite=True) for i, row in tt.iterrows())
[os.path.abspath(f) for f in files]

Wall time: 1min 4s


['M:\\PAG2019\\EPCExtent_30cm\\Orthos_Segmented_Classifiedv3\\W1004789_W449850_TrainingStackV3_MultiClassOptunaTunerGBLM_20211129_v2.tif',
 'M:\\PAG2019\\EPCExtent_30cm\\Orthos_Segmented_Classifiedv3\\W1004789_W444850_TrainingStackV3_MultiClassOptunaTunerGBLM_20211129_v2.tif',
 'M:\\PAG2019\\EPCExtent_30cm\\Orthos_Segmented_Classifiedv3\\W989789_W439850_TrainingStackV3_MultiClassOptunaTunerGBLM_20211129_v2.tif',
 'M:\\PAG2019\\EPCExtent_30cm\\Orthos_Segmented_Classifiedv3\\W979789_W419850_TrainingStackV3_MultiClassOptunaTunerGBLM_20211129_v2.tif',
 'M:\\PAG2019\\EPCExtent_30cm\\Orthos_Segmented_Classifiedv3\\W919789_W404850_TrainingStackV3_MultiClassOptunaTunerGBLM_20211129_v2.tif',
 'M:\\PAG2019\\EPCExtent_30cm\\Orthos_Segmented_Classifiedv3\\W1004789_W344850_TrainingStackV3_MultiClassOptunaTunerGBLM_20211129_v2.tif']

In [67]:
files

['../EPCExtent_30cm/Orthos_Segmented_Classifiedv3\\W1004789_W449850_TrainingStackV3_MultiClassOptunaTunerGBLM_20211129_v2.tif',
 '../EPCExtent_30cm/Orthos_Segmented_Classifiedv3\\W1004789_W444850_TrainingStackV3_MultiClassOptunaTunerGBLM_20211129_v2.tif',
 '../EPCExtent_30cm/Orthos_Segmented_Classifiedv3\\W989789_W439850_TrainingStackV3_MultiClassOptunaTunerGBLM_20211129_v2.tif',
 '../EPCExtent_30cm/Orthos_Segmented_Classifiedv3\\W979789_W419850_TrainingStackV3_MultiClassOptunaTunerGBLM_20211129_v2.tif',
 '../EPCExtent_30cm/Orthos_Segmented_Classifiedv3\\W919789_W404850_TrainingStackV3_MultiClassOptunaTunerGBLM_20211129_v2.tif',
 '../EPCExtent_30cm/Orthos_Segmented_Classifiedv3\\W1004789_W344850_TrainingStackV3_MultiClassOptunaTunerGBLM_20211129_v2.tif']

In [17]:
tt = tindex[(tindex.path == "W984789") & (tindex.row == "W454850")]

In [20]:
cFile = createClassifiedRaster(multiClassModel,
                       ortho_file=tt.OrthoFile.values[0],
                       classifiedFiles_loc=classifiedFiles_loc,
                       overwrite=True)
print(os.path.abspath(cFile))

%chime

PATH: W984789, ROW: W454850
Classified to M:\PAG2019\EPCExtent_30cm\Orthos_Segmented_Classifiedv3\W984789_W454850_TrainingStackV3_MultiClassOptunaTunerGBLM_20211115_v2.tif. 
Classification took 0:04:54.955734
M:\PAG2019\EPCExtent_30cm\Orthos_Segmented_Classifiedv3\W984789_W454850_TrainingStackV3_MultiClassOptunaTunerGBLM_20211115_v2.tif


In [ ]:
for i, row in tt.iterrows():
    createClassifiedRaster(multiClassModel,
                           ortho_file=row.OrthoFile,
                           classifiedFiles_loc=classifiedFiles_loc,
                           overwrite=True)
    break

In [ ]:
files = Parallel(n_jobs=3)(delayed(createClassifiedRaster)(multiClassModel,
                                                           ortho_file=row.OrthoFile,
                                                           classifiedFiles_loc=classifiedFiles_loc,
                                                           overwrite=True) for i, row in tt.iterrows())

In [4]:
#classifiers["PondLake"] = r"M:\PAG2019\notebooks\Models\lightGBMBinaryPondLake_OptunaTuner_20210903.sav"
#classifiers["Pool"] = r"M:\PAG2019\notebooks\Models\lightGBMBinaryPool_LGBMTuner_20211029.sav"
#classifiers["Asphault"] =  r"M:\PAG2019\notebooks\Models\lightGBMBinaryAsphault_LGBMTuner_20211029.sav"
classifiers["AllClassesOptimum"] = r"M:\PAG2019\notebooks\Models\lightGBM_20211104_optimum.sav"

testTarget = "AllClassesOptimum"

tt = tindex[((tindex.path == "W989789") & (tindex.row == "W439850")) |
            ((tindex.path == "W1004789") & (tindex.row == "W449850")) |
            ((tindex.path == "W1004789") & (tindex.row == "W344850")) |
            ((tindex.path == "W919789") & (tindex.row == "W404850")) 
           ]

for lcClass, model_loc in classifiers.items():
    if lcClass != testTarget:
        continue
    print(lcClass, model_loc)
    classifiedFiles = []
    lgbm_model = pickle.load(open(model_loc, 'rb'))
    if "AllClasses" in lcClass:
        t = classifyTiles(tt, lgbm_model, forceStart=True, binaryClass=None, overwrite=True)
    else:
        t = classifyTiles(tt, lgbm_model, forceStart=True, binaryClass=lcClass, overwrite=True)
        
    classifiedFiles += t
    
    print(classifiedFiles)


%chime

AllClassesOptimum M:\PAG2019\notebooks\Models\lightGBM_20211104_optimum.sav


Starting processing for 4 tiles

Starting W1004789_W449850 @ 2021-11-04 23:18:37.076686
PATH: W1004789, ROW: W449850
Classified to M:\PAG2019\EPCExtent_30cm\Orthos_Segmented_Classifiedv3\W1004789_W449850_TrainingStackV3_MultiClassOptunaTunerGBLM_20211104_v2.tif. 
Classification took 0:07:38.509692
Starting W989789_W439850 @ 2021-11-04 23:26:16.833379
PATH: W989789, ROW: W439850
Classified to M:\PAG2019\EPCExtent_30cm\Orthos_Segmented_Classifiedv3\W989789_W439850_TrainingStackV3_MultiClassOptunaTunerGBLM_20211104_v2.tif. 
Classification took 0:07:48.038113
Starting W919789_W404850 @ 2021-11-04 23:34:06.154468
PATH: W919789, ROW: W404850


KeyboardInterrupt: 

In [51]:
with rio.open(r"M:\PAG2019\EPCExtent_30cm\Orthos_Segmented_Classifiedv3\W989789_W439850_TrainingStackV3_MultiClassOptunaTunerGBLM.tif") as src:
    t = src.read(1)

uniques, counts = np.unique(t, return_counts=True) 
tt = dict(zip(uniques, counts))

pprint({k:round(100*vv/sum(counts),2) for kk,vv in  tt.items() for k,v in values_2019_lookup.items() if v ==kk})

{'PondLake': 0.1, 'Pool': 1.44, 'Barren': 14.21, 'SparseVeg': 3.4, 'DenseVeg': 6.19, 'IrrigatedLand': 0.15, 'Asphault': 21.94, 'ParkingLot': 21.94, 'Impervious': 34.45, 'Sidewalk': 34.45, 'Structure': 18.14}


### Cleanup

In [40]:
for file in classifiedFiles:
    cleanedFile = os.path.join(r"M:\PAG2019\EPCExtent_30cm\Orthos_Segmentedv3_Classifiedv3_cleaned", os.path.basename(file).replace(".tif", "_cleaned.tif"))
    try:
        with rio.open(file) as src:
            kwargs = src.profile
            data = src.read(1)
        with rio.open(cleanedFile, "w", **kwargs) as dst:
            dst.write(cleanupAsphault(data).astype(np.uint8), 1)
        print(f"Cleaned up file:\n\t{cleanedFile}")
    except:
        print(f"FAILED FOR: {cleanedFile}")
    

FAILED FOR: M:\PAG2019\EPCExtent_30cm\Orthos_Segmentedv3_Classifiedv3_cleaned\W1004789_W449850_TrainingStackV3_AsphaultBinaryLGBNewishGBLM_cleaned.tif
Cleaned up file:
	M:\PAG2019\EPCExtent_30cm\Orthos_Segmentedv3_Classifiedv3_cleaned\W989789_W439850_TrainingStackV3_AsphaultBinaryLGBNewishGBLM_cleaned.tif


In [49]:
buildVRT("../UrbanExtent_15cm/Elevation_40cmNPS/DSM40cm", "DSM40cm2019.vrt")

Created M:\PAG2019\UrbanExtent_15cm\Elevation_40cmNPS\DSM40cm\DSM40cm2019.vrt


In [ ]:
def cleanupBinary(array, binaryClass=None):
    if binaryClass == "Structure":
        out[(out==9) & (bands["HAG"]<5)] = 0 # if classified as structure, but less than 5 feet high, reclass to impervious
        #out[(bands["REDness"]>=35336) & (bands["GREENness"]<=34307) & (bands["BLUEness"]<=33409) & (lulc!=9) & (bands["DPR"]<=3)] = 3 # good for red bare earth
        out[(out==33023) & (bands["BLUEness"]<38547) & (bands["NIRness"]>23769)] = 0 # remove pools that aren't very blue AND does not have low NIRness value
            
        out[(bands["MSAVI"]>=29556) & (bands["GREENness"]>=34307) & (bands["RGBNmean"]<=19273) ] = 0 # set very green veg with high index and low brightness to irrigated
        
        out = smoothStructures(out) # smooth structures
        
for